In [1]:
import urllib
import numpy as np
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
from tqdm import tqdm
import plotly.express as px
import statsmodels.api as sm

In [2]:
stock = []

In [3]:
def RGBZCY(date):
    url = 'https://www.cbr.ru/hd_base/zcyc_params/zcyc/?DateTo='+date
    response = urllib.request.urlopen(url)
    myfile = response.read()
    file = open('output.txt', 'wb')
    file.write(myfile)
    file.close()
    with open('output.txt', 'br') as f:
        lines = list(f.readlines())[265:294]
    del lines[0:2]
    del lines[12:15]
    periods, yields = [], []
    periods = [float(str(lines[i]).replace("</th>\\r\\n'", "").replace("b'        <th>","")) for i in range(12)]
    yields = [float(str(lines[i]).replace("</td>\\r\\n'", "").replace("b'        <td>","")) for i in range(12,24)]
    table = pd.DataFrame({'Period': periods, 'Yield': yields}).set_index('Period')
    return table

def moex_data(ticker, num_of_years, is_index=None):
    startdate = str(dt.datetime.today().year-num_of_years)+'-'+str(dt.datetime.today().month)+'-'+str(dt.datetime.today().day-2)
    mgWeb = web.DataReader(ticker, 'moex', startdate, dt.datetime.today())
    if is_index == True:
        mgWeb = mgWeb[mgWeb['BOARDID']=='SNDX']
        mgWeb = mgWeb[['CLOSE']].reset_index()
    elif is_index == False:
        mgWeb = mgWeb[(mgWeb['BOARDID']=='RPMA')|(mgWeb['BOARDID']=='RPMO')]
        mgWeb = mgWeb[['CLOSE']].reset_index()
    if num_of_years > 1:
        annual_returns = [np.nan for i in range(0, 254)]
        annual_returns.extend([((mgWeb['CLOSE'][i]-mgWeb['CLOSE'][i-254])/mgWeb['CLOSE'][i-254])*100 for i in range(254, len(mgWeb))])
        mgWeb['AR,%']=annual_returns
    else:
        pass
    daily_returns=[0]
    daily_returns.extend([((mgWeb['CLOSE'][i]-mgWeb['CLOSE'][i-1])/mgWeb['CLOSE'][i-1])*100 for i in range(1, len(mgWeb))])
    mgWeb['DR,%']=daily_returns
    mgWeb['MONTH']=[dt.datetime.strftime(mgWeb['TRADEDATE'][i], '%Y-%m') for i in range(len(mgWeb))]
    return mgWeb

def month_data(DataFrame):
    monthly_returns = [np.nan]
    for i in range(1,len(DataFrame)):
        mr = (DataFrame['CLOSE'][i]-DataFrame['CLOSE'][i-1])/DataFrame['CLOSE'][i-1]*100
        monthly_returns.append(mr)
    DataFrame['MR,%'] = monthly_returns
    return DataFrame

def volatility(DataFrame, column):
    returns = [DataFrame[column][i]/100 for i in range(1, len(DataFrame))]
    return np.std(returns)

def beta_coef(stock, index, column):
    stock_returns = [stock[column][i]/100 for i in range(1, len(stock))]
    index_returns = [index[column][i]/100 for i in range(1, len(index))]
    min_len = min(len(stock_returns), len(index_returns))
    return (np.cov([stock_returns[i] for i in range(len(stock_returns)-min_len, len(stock_returns))], 
                  [index_returns[i] for i in range(len(index_returns)-min_len, len(index_returns))])[0,1])/((volatility(index))**2)

def rel_two_assets(stock_1, stock_2, column):
    asset_1 = [stock_1[column][i]/100 for i in range(1, len(stock_1))]
    asset_2 = [stock_2[column][i]/100 for i in range(1, len(stock_2))]
    min_len = min(len(asset_1), len(asset_2))
    cov = np.cov([asset_1[i] for i in range(len(asset_1)-min_len, len(asset_1))], 
                 [asset_2[i] for i in range(len(asset_2)-min_len, len(asset_2))])[0,1]
    corr = np.corrcoef([asset_1[i] for i in range(len(asset_1)-min_len, len(asset_1))], 
                       [asset_2[i] for i in range(len(asset_2)-min_len, len(asset_2))])[0,1]
    return cov, corr

def portfolio_two_stocks(stock_1, stock_2, w1, w2, column):
    ret_1 = np.mean([stock_1[column][i] for i in range(1,len(stock_1))])
    ret_2 = np.mean([stock_2[column][i] for i in range(1,len(stock_2))])
    vol_1 = volatility(stock_1, column)
    vol_2 = volatility(stock_2, column)
    cov, corr = rel_two_assets(stock_1, stock_2, column)
    portfolio_return = w1*ret_1+w2*ret_2
    portfolio_std = (w1**2*vol_1**2+w2**2*vol_2**2+2*w1*w2*corr*vol_1*vol_2)
    return portfolio_std, portfolio_return

def portfolio_table(stock_1, stock_2, column, p, date):
    w1 = np.linspace(0,1,p)
    w2 = np.array([1-i for i in w1])
    table = pd.DataFrame(columns = ['Volatility', 'Return'])
    volatilities, returns = [], []
    for i in tqdm(range(p)):
        std, ret = portfolio_two_stocks(stock_1, stock_2, w1[i], w2[i], column)
        volatilities.append(std)
        returns.append(ret)
    table['Volatility'] = volatilities
    table['Return'] = returns
    r_f = ((1+RGBZCY(date)['Yield'][1.00]/100)**(1/12)-1)*100 # in %
    table['Sharp_ratio'] = [(table['Return'][i]/100-r_f/100)/table['Volatility'][i] for i in range(len(table))]
    table['w1'] = w1
    table['w2'] = w2
    return table

def plot_portfolio(DataFrame):
    fig = px.scatter(DataFrame, x='Volatility', y='Return', color='Sharp_ratio')
    fig.show()

def plot_sharp(DataFrame):
    fig = px.scatter_3d(DataFrame, x='w1', y='w2', z='Sharp_ratio')
    fig.show()

In [6]:
plot_portfolio(portfolio_table(month_data(moex_data('YNDX', 2, False).groupby('MONTH').last()), 
                               month_data(moex_data('VTBR', 2, False).groupby('MONTH').last()), 
                               'MR,%', 
                               1000, 
                               '20.12.2019'))

100%|██████████| 1000/1000 [00:05<00:00, 174.12it/s]


In [5]:
plot_sharp(portfolio_table(month_data(moex_data('ALRS', 2, False).groupby('MONTH').last()), 
                           month_data(moex_data('VTBR', 2, False).groupby('MONTH').last()), 
                           'MR,%', 
                           1000, 
                           '20.12.2019'))

100%|██████████| 1000/1000 [00:04<00:00, 224.84it/s]


In [54]:
Y = list(moex_data('SBER', 2, False)['DR,%'])[1:]
X = list(moex_data('IMOEX', 2, True)['DR,%'])[1:]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1706
Date:                Fri, 27 Dec 2019   Prob (F-statistic):              0.680
Time:                        18:35:32   Log-Likelihood:                -3036.2
No. Observations:                 507   AIC:                             6076.
Df Residuals:                     505   BIC:                             6085.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6898      4.310      1.320      0.187      -2.778      14.158
x1            -1.9533      4.730     -0.413      0.680     -11.246       7.339
==============================================================================
Omnibus:                     1126.387   Durbin-Watson:                   2.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2306151.280
Skew:                          17.695   Prob(JB):                         0.00
Kurtosis:                     331.503   Cond. No.                         1.13
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
Y = list(month_data(moex_data('SBER', 2, False).groupby('MONTH').last())['MR,%'])[1:]
X = list(month_data(moex_data('IMOEX', 2, True).groupby('MONTH').last())['MR,%'])[1:]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     19.88
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           0.000197
Time:                        18:40:57   Log-Likelihood:                -75.008
No. Observations:                  24   AIC:                             154.0
Df Residuals:                      22   BIC:                             156.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1152      1.334     -1.586      0.127      -4.881       0.651
x1             1.8018      0.404      4.458      0.000       0.964       2.640
==============================================================================
Omnibus:                        7.612   Durbin-Watson:                   2.229
Prob(Omnibus):                  0.022   Jarque-Bera (JB):                5.446
Skew:                          -0.941   Prob(JB):                       0.0657
Kurtosis:                       4.380   Cond. No.                         3.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""